In [1]:
# Import libraries
import numpy as np
import pandas as pd
import re
import torchtext
import torch
from sklearn.model_selection import train_test_split
print(torchtext.__version__)

0.10.0


In [2]:
# Read input dataset
df = pd.read_csv('twitter_us_airline_sentiment/Tweets.csv')
print(df.shape)
df.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
# Drop most of the columns since we try to detect sentiment based only on text
df.drop(['tweet_id', 'airline_sentiment_confidence', 'negativereason', 'negativereason_confidence', 'airline', 'airline_sentiment_gold', 'name', 'negativereason_gold', 'retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone'], 1 , inplace=True)
print(df.shape)
df.head()

(14640, 2)


,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [4]:
# Unique categorical column values
print(" Sentiments: ", list(df['airline_sentiment'].unique()))

 Sentiments:  ['neutral', 'positive', 'negative']


In [5]:
# Label distribution of tweets
df['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [6]:
# Map the labels to integer values
df['airline_sentiment'] = df['airline_sentiment'].map({'negative':1, 'neutral':2, 'positive':3})
df.head()

,airline_sentiment,text
0,2,@VirginAmerica What @dhepburn said.
1,3,@VirginAmerica plus you've added commercials t...
2,2,@VirginAmerica I didn't today... Must mean I n...
3,1,@VirginAmerica it's really aggressive to blast...
4,1,@VirginAmerica and it's a really big bad thing...


In [7]:
# Dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  14640 non-null  int64 
 1   text               14640 non-null  object
dtypes: int64(1), object(1)
memory usage: 228.9+ KB


In [8]:
# Numerical columns statistics
df.describe()

,airline_sentiment
count,14640.000000
mean,1.534495
std,0.756084
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,3.000000


In [9]:
# Sample text values
list(df['text'].values)[:5]

['@VirginAmerica What @dhepburn said.',
 "@VirginAmerica plus you've added commercials to the experience... tacky.",
 "@VirginAmerica I didn't today... Must mean I need to take another trip!",
 '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
 "@VirginAmerica and it's a really big bad thing about it"]

In [10]:
# Remove hash tags function
def remove_hashtag(text):
    text = re.sub(r'@\w+','',text)
    return text

In [11]:
# Remove hash tag from text values
df['text'] = df['text'].map(remove_hashtag)
df.head(10)

,airline_sentiment,text
0,2,What said.
1,3,plus you've added commercials to the experien...
2,2,I didn't today... Must mean I need to take an...
3,1,"it's really aggressive to blast obnoxious ""en..."
4,1,and it's a really big bad thing about it
5,1,seriously would pay $30 a flight for seats th...
6,3,"yes, nearly every time I fly VX this “ear wor..."
7,2,Really missed a prime opportunity for Men Wit...
8,3,"Well, I didn't…but NOW I DO! :-D"
9,3,"it was amazing, and arrived an hour early. Yo..."


In [12]:
# Split into stratified train and test data
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['airline_sentiment'].values, stratify=df['airline_sentiment'].values, test_size=0.20, random_state=2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(11712,) (2928,) (11712,) (2928,)


In [13]:
# Create train and test dataframes
train_df = pd.DataFrame({'airline_sentiment': y_train, 'text': X_train})
test_df = pd.DataFrame({'airline_sentiment': y_test, 'text': X_test})
print(train_df.shape, test_df.shape)

(11712, 2) (2928, 2)


In [14]:
# Print sample train df
train_df.head()

,airline_sentiment,text
0,1,"called last night, after 10hrs got call back,..."
1,3,"thanks, i made it."
2,1,3 + hrs w/out bags being returned to us after...
3,1,- my son left his tablet on plane. Am out of...
4,2,thanks!!


In [15]:
# Check if train df contains proper proportions of sentiments 
train_df['airline_sentiment'].value_counts()

1    7342
2    2479
3    1891
Name: airline_sentiment, dtype: int64

In [16]:
# Check if test df contains proper proportions of sentiments 
test_df['airline_sentiment'].value_counts()

1    1836
2     620
3     472
Name: airline_sentiment, dtype: int64

In [17]:
# Save train and test df
train_df.to_csv('train_dataset.csv', index=False, header=False)
test_df.to_csv('test_dataset.csv', index=False, header=False)

In [18]:
# Create RawTextIterableDataset
from torchtext.data.datasets_utils import (
    _RawTextIterableDataset,
    _wrap_split_argument,
    _add_docstring_header,
    _create_dataset_directory,
    _create_data_from_csv,
)

def twitter_data(dataset_path, num_lines, dataset_name):
    DATASET_NAME = dataset_name
    NUM_LINES = num_lines
    path = dataset_path
    return _RawTextIterableDataset(DATASET_NAME, NUM_LINES,
                                   _create_data_from_csv(path))
  
tweet_data_iter = twitter_data('train_dataset.csv', 11712, 'train_dataset')

In [19]:
# Check the length of data loaded
len(tweet_data_iter)

11712

In [20]:
# Build vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(tweet_data_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [21]:
vocab(['here', 'is', 'an', 'example'])

[141, 16, 45, 1405]

In [22]:
# Length of train vocab
len(vocab)

14429

In [23]:
# Text and label processing pipelines
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [24]:
# Generate data batch and iterator
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

tweet_data_iter = twitter_data('train_dataset.csv', 11712, 'train_dataset')
dataloader = DataLoader(tweet_data_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)
len(dataloader)

1464

In [25]:
# Define a model
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [26]:
# Initiailize the model
tweet_data_iter = twitter_data('train_dataset.csv', 11712, 'train_dataset')
num_class = len(set([label for (label, text) in tweet_data_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [27]:
# Define train and evaluate functions
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [28]:
# Split dataset and execute the model
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 8 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_data_iter = twitter_data('train_dataset.csv', 11712, 'train_dataset')
test_data_iter = twitter_data('test_dataset.csv', 2928, 'test_dataset')
train_dataset = to_map_style_dataset(train_data_iter)
test_dataset = to_map_style_dataset(test_data_iter)
num_train = int(len(train_dataset) * 0.90)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1318 batches | accuracy    0.705
| epoch   1 |  1000/ 1318 batches | accuracy    0.747
-----------------------------------------------------------
| end of epoch   1 | time:  2.12s | valid accuracy    0.757 
-----------------------------------------------------------
| epoch   2 |   500/ 1318 batches | accuracy    0.785
| epoch   2 |  1000/ 1318 batches | accuracy    0.782
-----------------------------------------------------------
| end of epoch   2 | time:  1.92s | valid accuracy    0.770 
-----------------------------------------------------------
| epoch   3 |   500/ 1318 batches | accuracy    0.813
| epoch   3 |  1000/ 1318 batches | accuracy    0.814
-----------------------------------------------------------
| end of epoch   3 | time:  1.98s | valid accuracy    0.773 
-----------------------------------------------------------
| epoch   4 |   500/ 1318 batches | accuracy    0.842
| epoch   4 |  1000/ 1318 batches | accuracy    0.835
-------------------------

In [29]:
# Infer on test data
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.795


In [30]:
ag_news_label = {1: "negative",
                 2: "neutral",
                 3: "positive"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "and it's a poker face"

model = model.to("cpu")

print("This is a %s sentiment" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a neutral sentiment
